In [ ]:
# @title Colab Environment Setup
# @markdown Run this cell to download the datasets from GitHub automatically.

import os
import requests
import time

# GitHub Repo Base URL (Raw content)
repo_url = "https://raw.githubusercontent.com/frzerxz/spotify-ml-analysis/main/"
files = ["dataset.csv", "spotify_data clean.csv", "Most Streamed Spotify Songs 2024.csv"]

print("Downloading datasets (Force Refresh)...")
timestamp = str(int(time.time()))
for file in files:
    # Always remove old file to ensure fresh download
    if os.path.exists(file):
        os.remove(file)
    
    # Add timestamp to bypass cache
    url = repo_url + file.replace(" ", "%20") + f"?v={timestamp}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(file, "wb") as f:
                f.write(response.content)
            print(f"✅ Downloaded: {file}")
        else:
            print(f"❌ Failed to download {file} (Status: {response.status_code})")
    except Exception as e:
        print(f"❌ Error downloading {file}: {e}")

print("\nInstalling necessary libraries...")
!pip install xgboost shap imbalanced-learn
print("Setup complete!")

# Makine Öğrenmesi Dönem Ödevi Projesi Raporu (F2025)

**Öğrenci Adı Soyadı:** Firuze Eroğlu  
**Öğrenci No:** 201613709044  
**Teslim Tarihi:** 14.12.2025  

---

## 1. Veri Seti Tanıtımı

Bu projede üç farklı makine öğrenmesi problemi için, müzik endüstrisinin en büyük platformu Spotify'dan elde edilen gerçek dünya verileri kullanılmıştır. Veri setleri Kaggle platformundan "Open Access" (Açık Erişim) lisansıyla temin edilmiştir. Aşağıda her bir veri setinin kaynağı, içeriği ve proje kapsamındaki kullanım amacı detaylandırılmıştır.

### 1.1. Sınıflandırma Veri Seti: Spotify Tracks Dataset
*   **Veri Kaynağı Linki:** [Kaggle - Spotify Tracks Dataset](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset)
*   **Veri Seti Tanımı:** Bu veri seti, Spotify üzerindeki 125 farklı müzik türünden yaklaşık 114.000 şarkıyı kapsamaktadır. Her bir satır bir şarkıyı temsil eder ve şarkıya ait teknik ses özelliklerini (audio features) içerir.
*   **Problem ve Tahmin Hedefi:**
    *   **Problem:** Müzik endüstrisinde bir şarkının "Hit" olup olmayacağını, şarkı henüz piyasaya çıkmadan sadece ses analizine dayanarak öngörmek büyük bir ticari değer taşır.
    *   **Hedef:** Şarkının `danceability` (dans edilebilirlik), `energy` (enerji), `loudness` (ses şiddeti), `acousticness` gibi teknik ses özniteliklerini kullanarak, şarkının **"Popüler"** (Popularity > 50) olup olmadığını sınıflandırmaya çalışıyoruz. Bu, **Binary Classification** (İkili Sınıflandırma) problemidir.
*   **Kullanılan Öznitelikler (Features):**
    *   `danceability`: Tempo, ritim kararlılığı ve vuruş gücüne göre şarkının dansa uygunluğu (0.0 - 1.0).
    *   `energy`: Şarkının yoğunluk ve aktivite ölçümü. Hızlı, gürültülü şarkılar 1.0'a yakındır.
    *   `valence`: Şarkının taşıdığı müzikal pozitiflik (mutluluk/hüzün) düzeyi.
    *   `instrumentalness`: Şarkının ne kadar enstrümantal olduğu (vokal olup olmadığı).
    *   `duration_ms`: Şarkının milisaniye cinsinden süresi.

### 1.2. Regresyon Veri Seti: Spotify Global Music Dataset
*   **Veri Kaynağı Linki:** [Kaggle - Spotify Global Music Dataset 2009-2025](https://www.kaggle.com/datasets/wardabilal/spotify-global-music-dataset-20092025?select=spotify_data+clean.csv) (Kullanılan dosya: `spotify_data clean.csv`)
*   **Veri Seti Tanımı:** 2009-2025 yılları arasındaki küresel müzik trendlerini içeren, temizlenmiş ve yapılandırılmış bir veri setidir. Yaklaşık 8.500 örnek içermektedir.
*   **Problem ve Tahmin Hedefi:**
    *   **Problem:** Bir şarkının başarısı sadece ses özelliklerine mi bağlıdır, yoksa sanatçının şöhreti ve albümün yapısı daha mı etkilidir?
    *   **Hedef:** Şarkının ve sanatçının metadatalarını (metadata) kullanarak, şarkının Spotify üzerindeki **Popülerlik Puanını (0 ile 100 arasında sürekli bir değer)** sayısal olarak tahmin etmek. Bu, bir **Regresyon** (Kestirim) problemidir.
*   **Kullanılan Öznitelikler (Features):**
    *   `artist_popularity`: Şarkıyı söyleyen sanatçının genel popülerlik skoru.
    *   `artist_followers`: Sanatçının takipçi sayısı (Sanatçının hayran kitlesi).
    *   `album_total_tracks`: Şarkının bulunduğu albümdeki toplam şarkı sayısı.
    *   `album_type`: Albümün türü (Single, Album, Compilation).
    *   `explicit`: Şarkının sansürlü/küfürlü içerik barındırıp barındırmadığı.

### 1.3. Kümeleme Veri Seti: Most Streamed Spotify Songs 2024
*   **Veri Kaynağı Linki:** [Kaggle - Most Streamed Spotify Songs 2024](https://www.kaggle.com/datasets/nelgiriyewithana/most-streamed-spotify-songs-2024)
*   **Veri Seti Tanımı:** 2024 yılının en çok dinlenen şarkılarını ve bu şarkıların farklı platformlardaki (Spotify, TikTok, YouTube, vb.) etkileşim sayılarını içeren güncel bir veri setidir.
*   **Problem ve Analiz Hedefi:**
    *   **Problem:** Milyonlarca dinlenen şarkılar arasında gizli örüntüler veya farklı başarı profilleri var mıdır? (Örneğin: "TikTok sayesinde ünlü olanlar" vs "Organik hitler").
    *   **Hedef:** Veri setindeki şarkıların **Etiketsiz** (Unsupervised) olarak benzerliklerine göre gruplandırılmasıdır. Şarkıları dinlenme sayıları, çalma listelerine eklenme sıklığı ve sosyal medya etkileşimlerine göre **Segmentlere (Kümeler)** ayırarak, müzik dünyasındaki farklı başarı tiplerini analiz etmeyi hedefliyoruz.
*   **Kullanılan Öznitelikler (Features):**
    *   `Spotify Streams`: Toplam dinlenme sayısı.
    *   `Spotify Playlist Count`: Şarkının kaç farklı çalma listesine eklendiği.
    *   `TikTok Views`: Şarkının TikTok platformundaki görüntülenme sayısı.
    *   `YouTube Views`: Şarkının YouTube görüntülenme sayısı.
    *   `AirPlay Spins`: Radyolarda çalınma sayısı.

---

## 2. Uygulanan Yöntemler ve Ön İşleme

### 2.1. Veri Ön İşleme (Preprocessing)
Tüm veri setleri için aşağıdaki standart adımlar uygulanmıştır:
*   **Eksik Veriler (Missing Values):** Sayısal sütunlardaki eksik veriler medyan ile, kategorik veriler sabit değer veya en sık görülen değer ile dolduruldu ('SimpleImputer').
*   **Özellik Ölçekleme (Scaling):** Tüm algoritmaların (özellikle K-Means ve Logistic Regression) performansını artırmak için `StandardScaler` ile veriler ölçeklendi (Ortalama=0, Varyans=1).
*   **Kategorik Dönüşüm:** `OneHotEncoder` kullanılarak 'album_type' ve 'explicit' gibi kategorik veriler sayısal matrise dönüştürüldü.
*   **Dengesiz Veri Çözümü (Undersampling):** Sınıflandırma görevinde, "Popüler Olmayan" şarkıların sayısı "Popüler" olanlardan çok daha fazlaydı. Modelin çoğunluk sınıfına meyletmesini önlemek için **RandomUnderSampler** kullanılarak çoğunluk sınıfındaki veri sayısı azaltıldı ve iki sınıf eşitlendi. Bu, özellikle 'Recall' değerini artırmak için kritik bir adımdı.
*   **Aykırı Değer Analizi (Outlier Handling):** Regresyon görevinde IQR (Interquartile Range) yöntemi kullanılarak veri setindeki aşırı uç değerler temizlendi.
*   **Veri Ayrımı ve Doğrulama (Validation):** Tüm görevlerde veri seti `%80 Eğitim - %20 Test` oranında ve `stratify` (sınıf dengesini koruyarak) parametresiyle ayrıldı. Ayrıca, **Random Forest** modeli eğitilirken **3-Katlı Çapraz Doğrulama (3-Fold Cross-Validation)** ("k-fold cross val") kullanıldı. Bu yöntem, modelin veriye aşırı uyum (overfitting) sağlamasını engelledi ve sonuçların güvenilirliğini artırdı.

### 2.2. Algoritma Seçimi
*   **Sınıflandırma:** Logistic Regression, Random Forest Classifier, XGBoost. (RF için `RandomizedSearchCV` ile optimizasyon yapıldı).
*   **Regresyon:** Linear Regression, Random Forest Regressor, Gradient Boosting.
*   **Kümeleme:** K-Means, MiniBatch K-Means, DBSCAN. (K değeri Silhouette analizi ile belirlendi).

---

## 3. Sonuçlar ve Metrikler

### 3.1. Sınıflandırma Sonuçları ve Görseller
Sınıflandırma görevinde "Popülerlik" tahmini için modellerin performansı karşılaştırılmıştır.

#### **A. Model Performansı ve Karşılaştırma**
Aşağıdaki grafik, üç modelin "Accuracy" (Doğruluk) ve "Recall" (Yakama) oranlarını göstermektedir.
![Model Karşılaştırması](model_comparison_chart.png)

*   **Yorum:** Grafikte görüldüğü üzere, **Random Forest (Optimized)** modeli en dengeli performansı sergilemiştir. Hiperparametre optimizasyonu öncesi "Popüler" sınıfını yakalamakta zorlanan model, optimizasyon ve undersampling sonrası her iki sınıfı da %75 civarında başarıyla tahmin edebilmiştir.

#### **B. Hata Matrisi (Confusion Matrix)**
Modelin nerede hata yaptığını anlamak için Random Forest modelinin Karmaşıklık Matrisi (Confusion Matrix) aşağıdadır:
![Confusion Matrix](cm_Random_Forest_(Optimized).png)

*   **Yorum:** Matris incelendiğinde, modelin "Popüler Olmayan" (0) şarkıların büyük çoğunluğunu doğru bildiği, ancak "Popüler" (1) şarkıların bir kısmını hala kaçırdığı görülmektedir. Müzik başarısının sadece ses özelliklerine bağlı olmaması (pazarlama, bütçe vb.) bu hatanın doğal sebebidir.

#### **C. Model Açıklanabilirliği (SHAP Analizi)**
Modelin *neden* bu kararı verdiğini anlamak için XGBoost modeli üzerinde SHAP analizi yapılmıştır:
![SHAP Analizi](shap_summary.png)

*   **Detaylı Analiz:**
    1.  **Loudness (Ses Şiddeti):** En belirleyici özellik. Grafikte sağa doğru (yüksek değerler) kırmızı noktaların yoğunlaşması, **daha gürültülü/yüksek sesli şarkıların popüler olma ihtimalinin daha yüksek olduğunu** göstermektedir.
    2.  **Acousticness:** Düşük akustik (daha elektronik/üretilmiş) şarkılar popülerlikle pozitif korelasyon göstermektedir.
    3.  **Duration:** Çok uzun şarkılar popülerlik şansını düşürmektedir.

### 3.2. Regresyon Sonuçları ve Görseller
Şarkı popülaritesini (0-100) tahmin etme başarısı:

#### **A. Tahmin Başarısı (Gerçek vs Tahmin)**
Random Forest modelinin tahminleri ile gerçek değerlerin karşılaştırması:
![Regresyon Tahminleri](reg_pred_Random_Forest.png)

*   **Yorum:** İdeal durumda tüm noktaların kırmızı köşegen çizgi üzerinde olması gerekirdi. Grafikteki saçılım, modelin genel trendi yakaladığını (R² ~ 0.24) ancak birebir puan tahmininde sapmalar yaşadığını gösterir. RMSE (Hata Kareler Ortalaması) yaklaşık 20 puandır; yani model bir şarkıya "60 puan" dediğinde, gerçek puan 40 veya 80 olabilir.

#### **B. Özellik Önemi (Feature Importance - Feature Selection)**
Hangi faktörler bir şarkının puanını artırır?
![Feature Importance](regression_feature_importance.png)

*   **Kritik Bulgular:**
    *   **Artist Popularity (Sanatçı Popülerliği):** Grafikteki en uzun çubuktur. Bu, şarkının *nasıl* duyulduğundan ziyade *kimin* söylediğinin başarıda en büyük etken olduğunu kanıtlar.
    *   **Artist Followers:** İkinci sırada sanatçının hayran kitlesi gelmektedir.
    *   **Explicit (Sansür):** Şarkının küfürlü olup olmamasının popülerlik üzerinde marjinal bir etkisi vardır.

### 3.3. Kümeleme Sonuçları ve Görseller (PCA ve Segmentasyon)
Şarkıların dinlenme ve etkileşim sayılarına göre gruplandırılması:

#### **A. Küme Dağılımı (K-Means, K=2) ve PCA**
Veri seti PCA (Principal Component Analysis) ile 2 boyuta indirgenmiş ve kümeler görselleştirilmiştir:
![K-Means Kümeleme](clustering_K-Means_(K=2).png)

*   **Küme Analizi:**
    *   **Küme 0 (Mor):** Düşük ve orta seviye dinlenmeye sahip şarkıların oluşturduğu ana gövde. Şarkıların %90'ı buradadır.
    *   **Küme 1 (Sarı):** "Süper Hit" şarkılar. Sayıları azdır ancak grafikte diğerlerinden net bir şekilde ayrışmışlardır.

#### **B. Platform Etkileşimi (Spotify vs TikTok)**
Şarkıların Spotify dinlenmeleri ile TikTok görüntülenmeleri arasındaki ilişki:
![Scatter Plot](clustering_scatter_streams.png)

*   **Yorum:** Grafikte, TikTok'ta çok görüntülenen şarkıların (Y ekseni yüksek), Spotify'da da çok dinlendiği (X ekseni yüksek) bariz bir **pozitif korelasyon** görülmektedir. Bu, sosyal medya viralitesinin müzik başarısındaki kilit rolünü doğrular.

---

## 4. Sonuçların Özeti (Summary of Results)

Kodların son çalıştırılması neticesinde elde edilen başarı metrikleri ve süreler aşağıdaki tabloda özetlenmiştir:

| Görev | Model | Ana Metrik | Değer | Eğitim Süresi (sn) |
| :--- | :--- | :--- | :--- | :--- |
| **Sınıflandırma** | Logistic Regression | Accuracy | %56 | 0.04 sn |
| | Random Forest (Opt) | Accuracy | %74 | 100.06 sn |
| | XGBoost | Accuracy | %74 | 2.63 sn |
| **Regresyon** | Linear Regression | R2 Score | 0.22 | 0.002 sn |
| | Random Forest | R2 Score | 0.24 | 0.71 sn |
| | Gradient Boosting | R2 Score | 0.24 | 0.50 sn |
| **Kümeleme** | K-Means (K=2) | Silhouette | 0.45 | 0.003 sn |
| | DBSCAN | Silhouette | 0.09 | 0.06 sn |

*(Not: Tablodaki değerler çalışılan bilgisayarın donanımına göre küçük değişiklikler gösterebilir.)*

---

## 5. Tartışma ve Yorumlar (Discussion)

Proje sonuçlarında elde edilen teknik bulguların yorumlanması:

1.  **Dengesiz Veri Yönetimi:** Sınıflandırma görevinde, veri seti dengesiz olduğu için başlangıçta model sadece çoğunluk sınıfını (Popüler Olmayan) tahmin ediyordu. `Undersampling` tekniği uygulandıktan sonra doğruluk (accuracy) düşmüş gibi görünse de, aslında *Recall* (Duyarlılık) metriği ciddi oranda artarak modelin "Popüler" şarkıları yakalama yeteneği kazandırıldı. Bu, gerçek hayatta "Hit Şarkı" avcılığı için daha doğru bir yaklaşımdır.
2.  **Özniteliklerin Gücü (Feature Relevance):** Regresyon analizi net bir şekilde gösterdi ki, bir şarkının popülerliği, o şarkının müzikal yapısından (bpm, key, mode) ziyade, sanatçının mevcut şöhreti (`artist_popularity`) ile ilişkilidir. Bu durum, müzik endüstrisinde "yıldız gücünün" içerikten daha baskın olabileceği tezini destekler.
3.  **Kümeleme Yapısı:** Müzik piyasası homojen bir dağılım göstermez. K-Means ve Silhouette analizi, verinin en iyi 2 kümeye ayrıldığını gösterdi: "Mega Hit'ler" ve "Diğerleri". Bu, endüstrideki gelir dağılımı eşitsizliğine (Pareto Prensibi) işaret eder.
4.  **Cross-Validation Etkisi:** Model eğitiminde sadece `Train-Test Split` yerine `Cross-Validation` kullanılması, özellikle küçük veri setlerinde (Regresyon verimiz 8.500 satırdı) modelin kararlılığını artırdı. Tek bir test setine bağlı kalmak yerine verinin farklı parçalarında test yapılması, elde ettiğimiz R² skorunun şans eseri olmadığını doğruladı.
5.  **Model Performansları:** Ağaç tabanlı modellerin (Random Forest ve XGBoost), doğrusal modellere (Linear/Logistic Regression) göre karmaşık ilişkileri modellemede daha başarılı olduğu gözlemlendi. Ancak bu modellerin eğitimi ve hiperparametre optimizasyonu (GridSearchCV) çok daha uzun sürdü (~100 sn vs ~0.1 sn).

---

## 6. Kazanımlar / Öğrenilenler (Learnings)

Bu dönem ödevi projesi kapsamında şunlar öğrenilmiştir:
*   **Veri Kalitesinin Önemi:** Gerçek dünya verilerinin "temiz" olmadığını, eksik verilerin (missing values) ve gürültünün (noise) model başarısını doğrudan etkilediği görüldü.
*   **Metriklerin Doğru Okunması:** Sadece `Accuracy`'ye bakmanın yanıltıcı olabileceği, dengesiz veri setlerinde `Recall` ve `F1-Score`'un hayati olduğu anlaşıldı.
*   **İş Bilgisi (Domain Knowledge):** Veri biliminde sadece kod yazmanın yetmediği, verinin ait olduğu alanın (müzik endüstrisi) dinamiklerini bilmenin (örn: TikTok'un etkisi) analiz başarısını artırdığı fark edildi.
*   **AI ve Otomasyon:** Hiperparametre optimizasyonu ve özellik seçimi gibi süreçlerin yapay zeka ve otomasyon araçlarıyla ne kadar hızlandırılabileceği deneyimlendi.

---

## 7. Akademik Dürüstlük ve Yapay Zekâ Kullanımı Beyanı

Bu ödevin hazırlanmasında;
*   **Kodlama:** Python scriptlerinin (`pandas`, `scikit-learn`, `imblearn` kullanımı) hazırlanması, hata ayıklama (debugging) ve grafik çizdirme süreçlerinde **AI Asistanı** (Antigravity/Gemini) desteği alınmıştır. AI, kodun iskeletini oluşturmada ve syntax hatalarını düzeltmede kullanılmıştır.
*   **Strateji:** Dengesiz veri setlerinde `recall` sorununu çözmek için `undersampling` yönteminin seçilmesi ve uygulanmasında AI rehberliğinden faydalanılmıştır. Ancak veri setlerinin seçimi ve problemin kurgulanması tarafımca yapılmıştır.
*   **Yorumlama:** Elde edilen teknik çıktıların (R² skoru, SHAP değerleri, Recall dengesi, Confusion Matrix yorumları) analizi ve raporlanması, kendi cümlelerimle ve derste öğrenilen bilgiler ışığında şahsım tarafından yapılmıştır.

---

### Ek 1: Çalışma Ortamı (Donanım Özellikleri)
Proje aşağıdaki donanım özelliklerine sahip bilgisayarda çalıştırılmıştır:
*   **Model:** Monster Abra A5 V15.7
*   **İşletim Sistemi:** Windows
*   **İşlemci (CPU):** Intel Core i5 10. Nesil
*   **Bellek (RAM):** 16 GB (DDR4)
*   **Ekran Kartı (GPU):** NVIDIA GeForce GTX 1650 Ti


# Classification Task
Kodlar aşağıdadır:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, classification_report)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import shap
import time

print("Veri Seti Yükleniyor (Spotify Dataset)...")
try:
    df = pd.read_csv('dataset.csv', encoding='latin1')
    print(f"Veri seti boyutu: {df.shape}")
except FileNotFoundError:
    print("Hata: 'dataset.csv' bulunamadı.")
    exit()

target_col = 'is_popular'
df[target_col] = (df['popularity'] > 50).astype(int)
print(f"Yeni hedef değişken: {target_col}")

cols_to_drop = ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'track_genre']
feature_cols = [c for c in df.columns if c not in cols_to_drop and c != target_col]
X = df[feature_cols]
y = df[target_col]

print(f"Kullandığım özellikler:\n{X.columns.tolist()}")

if 'explicit' in X.columns:
    X['explicit'] = X['explicit'].astype(int)

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category', 'bool']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Train boyutu: {X_train.shape}, Sınıf Dağılımı: {np.bincount(y_train)}")

try:
    from imblearn.under_sampling import RandomUnderSampler
    rus = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
    print(f"Resampled train boyutu: {X_train_resampled.shape}, Sınıf Dağılımı: {np.bincount(y_train_resampled)}")
    X_train = X_train_resampled
    y_train = y_train_resampled
except ImportError:
    print("UYARI: 'imblearn' kütüphanesi yok. Manuel undersampling uyguluyorum...")
    train_df = pd.concat([X_train, y_train], axis=1)
    minority_count = y_train.value_counts().min()
    df_0 = train_df[train_df[target_col] == 0].sample(minority_count, random_state=42)
    df_1 = train_df[train_df[target_col] == 1]
    balanced_df = pd.concat([df_0, df_1])
    X_train = balanced_df.drop(columns=[target_col])
    y_train = balanced_df[target_col]
    print(f"Manuel Resampled Boyut: {X_train.shape}")

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("\nAykırı değerleri (Outliers) inceliyorum...")
try:
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    if 'duration_ms' in df.columns:
        sns.boxplot(x=df['duration_ms'] / 60000)
        plt.title('Şarkı Süresi Dağılımı (Dakika)')
        plt.xlabel('Süre (dk)')

    plt.subplot(1, 2, 2)
    if 'danceability' in df.columns:
        sns.histplot(df['danceability'], bins=30, kde=True)
        plt.title('Dans Edilebilirlik Dağılımı')
    else:
        sns.histplot(df.iloc[:, 0], bins=30)
        plt.title('Birinci Sütun Dağılımı')

    plt.tight_layout()
    plt.savefig('classification_outliers.png')
    plt.close()
except Exception as e:
    print(f"Grafik hatası: {e}")

print("\nModelleri eğitiyorum...")

results = []

def evaluate_model(name, model, X_tr, y_tr, X_te, y_te):
    start_time = time.time()
    model.fit(X_tr, y_tr)
    train_time = time.time() - start_time

    y_pred = model.predict(X_te)
    y_prob = model.predict_proba(X_te)[:, 1] if hasattr(model, "predict_proba") else np.zeros(len(y_te))

    acc = accuracy_score(y_te, y_pred)
    prec = precision_score(y_te, y_pred, average='weighted')
    rec = recall_score(y_te, y_pred, average='weighted')
    f1 = f1_score(y_te, y_pred, average='weighted')

    report_dict = classification_report(y_te, y_pred, output_dict=True)
    recall_0 = report_dict['0']['recall']
    recall_1 = report_dict['1']['recall']

    try:
        auc = roc_auc_score(y_te, y_prob)
    except:
        auc = 0.5

    print(f"--- {name} Sonuçları ---")
    print(f"Bütün: Accuracy: {acc:.4f}, AUC: {auc:.4f}")
    print(f"Recall (Popüler Olmayan): {recall_0:.4f}")
    print(f"Recall (Popüler): {recall_1:.4f}")
    print(classification_report(y_te, y_pred))

    cm = confusion_matrix(y_te, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {name}')
    plt.ylabel('Gerçek')
    plt.xlabel('Tahmin')
    plt.savefig(f'cm_{name.replace(" ", "_")}.png')
    plt.close()

    return {
        'Model': name, 'Accuracy': acc, 'Precision': prec,
        'Recall': rec, 'F1': f1, 'AUC': auc,
        'Recall_0': recall_0, 'Recall_1': recall_1,
        'Time (s)': train_time
    }

lr_model = LogisticRegression(max_iter=1000)
results.append(evaluate_model("Logistic Regression", lr_model, X_train_processed, y_train, X_test_processed, y_test))

print("\nRandom Forest için en iyi parametreleri arıyorum...")
rf = RandomForestClassifier(random_state=42)

param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=5,
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

start_search = time.time()
random_search.fit(X_train_processed, y_train)
print(f"Bulunan en iyi parametreler: {random_search.best_params_}")
print(f"Arama süresi: {time.time() - start_search:.2f} sn")

best_rf = random_search.best_estimator_
results.append(evaluate_model("Random Forest (Optimized)", best_rf, X_train_processed, y_train, X_test_processed, y_test))

print("\nXGBoost...")
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
results.append(evaluate_model("XGBoost", xgb_model, X_train_processed, y_train, X_test_processed, y_test))

results_df = pd.DataFrame(results)
print("\nKarşılaştırma Tablosu:")
print(results_df)
results_df.to_csv('model_comparison_classification.csv', index=False)

plt.figure(figsize=(10, 6))
results_df.set_index('Model')[['Accuracy', 'Recall_1', 'AUC']].plot(kind='bar', figsize=(10, 6))
plt.title('Model Performansı (Accuracy vs Popüler Şarkı Yakalama)')
plt.ylabel('Skor')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('model_comparison_chart.png')

print("\nSHAP ile model analizi...")
try:
    explainer = shap.TreeExplainer(xgb_model)
    X_shap = X_test_processed[:100].toarray() if hasattr(X_test_processed, "toarray") else X_test_processed[:100]
    shap_values = explainer.shap_values(X_shap)
    plt.figure()
    shap.summary_plot(shap_values, X_shap, show=False, feature_names=numeric_features.tolist())
    plt.tight_layout()
    plt.savefig('shap_summary.png')
except Exception as e:
    print(f"SHAP hatası: {e}")

print("\nSınıflandırma işlemi bitti.")


# Regression Task
Kodlar aşağıdadır:

In [ ]:

import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import time

print("Veri Seti Yükleniyor (Spotify Data)...")
try:
    df = pd.read_csv('spotify_data clean.csv', encoding='latin1')
    print(f"Veri seti boyutu: {df.shape}")
except FileNotFoundError:
    print("Hata: 'spotify_data clean.csv' bulunamadı.")
    exit()

target_col = 'track_popularity'

cols_to_drop = ['track_id', 'track_name', 'artist_name', 'album_id', 'album_name', 'album_release_date', 'artist_genres', target_col]

X = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
y = df[target_col]

print("Kullandığım özellikler:", X.columns.tolist())

print("\nVeri ön işleme adımları...")

print("\nVeri ön işleme adımları...")

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'bool']).columns

Q1 = df[numeric_features].quantile(0.25)
Q3 = df[numeric_features].quantile(0.75)
IQR = Q3 - Q1

condition = ~((df[numeric_features] < (Q1 - 3 * IQR)) | (df[numeric_features] > (Q3 + 3 * IQR))).any(axis=1)
original_len = len(df)
df_clean = df[condition]
X = df_clean.drop(columns=[c for c in cols_to_drop if c in df_clean.columns])
y = df_clean[target_col]

print(f"Outlier temizliğinden sonra kalan veri: {len(df_clean)} (Atılan: {original_len - len(df_clean)})")

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train: {X_train.shape}, Test: {X_test.shape}")

X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

results = []

def evaluate_regressor(name, model, X_tr, y_tr, X_te, y_te):
    start_time = time.time()
    model.fit(X_tr, y_tr)
    train_time = time.time() - start_time

    y_pred_tr = model.predict(X_tr)
    y_pred_te = model.predict(X_te)

    r2 = r2_score(y_te, y_pred_te)
    rmse = np.sqrt(mean_squared_error(y_te, y_pred_te))
    mae = mean_absolute_error(y_te, y_pred_te)

    print(f"--- {name} ---")
    print(f"R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, Süre: {train_time:.2f}sn")

    plt.figure(figsize=(6, 6))
    plt.scatter(y_te, y_pred_te, alpha=0.3)
    plt.plot([y_te.min(), y_te.max()], [y_te.min(), y_te.max()], 'r--')
    plt.xlabel('Gerçek Değerler')
    plt.ylabel('Tahminler')
    plt.title(f'{name}: Gerçek vs Tahmin')
    plt.savefig(f'reg_pred_{name.replace(" ", "_")}.png')
    plt.close()

    return {'Model': name, 'R2': r2, 'RMSE': rmse, 'MAE': mae, 'Training Time': train_time}

lr = LinearRegression()
results.append(evaluate_regressor("Linear Regression", lr, X_train_scaled, y_train, X_test_scaled, y_test))

rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
results.append(evaluate_regressor("Random Forest", rf, X_train_scaled, y_train, X_test_scaled, y_test))

gb = GradientBoostingRegressor(n_estimators=100, random_state=42)
results.append(evaluate_regressor("Gradient Boosting", gb, X_train_scaled, y_train, X_test_scaled, y_test))

results_df = pd.DataFrame(results)
print("\nKarşılaştırma Sonuçları:")
print(results_df)

print("\nHangi özellikler önemli? (Feature Importance)")
try:

    num_names = numeric_features.tolist()

    try:
        cat_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features).tolist()
    except:
        cat_names = [f"Cat_{i}" for i in range(X_train_scaled.shape[1] - len(num_names))]

    all_features = num_names + cat_names

    if len(all_features) != X_train_scaled.shape[1]:
        print("Uyarı: Özellik isimleri ile veri boyutu uyuşmuyor, grafik çizilmeyecek.")
    else:
        feature_importances = rf.feature_importances_
        indices = np.argsort(feature_importances)[::-1][:10]

        plt.figure(figsize=(10, 6))
        plt.title("En Önemli 10 Özellik (Random Forest)")
        plt.bar(range(10), feature_importances[indices], align="center")
        plt.xticks(range(10), [all_features[i] for i in indices], rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('regression_feature_importance.png')
        print("Özellik önem grafiği kaydedildi.")
except Exception as e:
    print(f"Feature importance grafiği çizilirken hata: {e}")

print("\nRegresyon işlemi bitti.")


# Clustering Task
Kodlar aşağıdadır:

In [ ]:

import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
import time

print("Veri Seti Yükleniyor (Most Streamed Spotify Songs 2024)...")
try:

    df = pd.read_csv('Most Streamed Spotify Songs 2024.csv', encoding='latin1')
    print(f"Veri Boyutu: {df.shape}")
    print(df.head())
except FileNotFoundError:
    print("Hata: 'Most Streamed Spotify Songs 2024.csv' dosyası bulunamadı!")
    exit()

drop_cols = ['Track', 'Album Name', 'Artist', 'ISRC', 'All Time Rank', 'Release Date']

cols_to_drop = [c for c in drop_cols if c in df.columns]
X = df.drop(columns=cols_to_drop)

for col in X.columns:

    X[col] = pd.to_numeric(X[col].astype(str).str.replace(',', ''), errors='coerce')

X = X.select_dtypes(include=[np.number])

X = X.dropna(axis=1, how='all')

X = X.fillna(0)

print(f"Özellik sayısı: {X.shape[1]}")
print(f"Özellikler: {X.columns.tolist()}")

if not np.isfinite(X).all().all():
    print("Sonsuz değerler temizleniyor...")
    X = X.replace([np.inf, -np.inf], 0)

print(f"Özellik sayısı: {X.shape[1]}")
print(f"Özellikler: {X.columns.tolist()}")

if len(X) > 10000:
    print("Veri çok büyük olduğu için ilk 5000 satırı alıyorum.
    X = X.iloc[:5000]

print("\nÖlçekleme...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

results = []

def evaluate_clustering(name, model_labels, X_data, time_taken):

    if len(np.unique(model_labels)) < 2:
        print(f"{name}: Yetersiz küme sayısı (tümü gürültü veya tek küme).")
        return None

    sil = silhouette_score(X_data, model_labels, sample_size=1000, random_state=42)
    db = davies_bouldin_score(X_data, model_labels)

    n_clusters = len(set(model_labels)) - (1 if -1 in model_labels else 0)
    print(f"--- {name} ---")
    print(f"Küme Sayısı: {n_clusters}")
    print(f"Silhouette: {sil:.4f}, Davies-Bouldin: {db:.4f}")

    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=X_pca[:, 0], y=X_pca[:,1], hue=model_labels, palette='viridis', legend='full', alpha=0.6)
    plt.title(f'{name} Sonuçları (PCA İndirgeme)')
    plt.savefig(f'clustering_{name.replace(" ", "_")}.png')
    plt.close()

    return {'Model': name, 'Clusters': n_clusters, 'Silhouette': sil, 'DB Index': db, 'Time': time_taken}

print("En iyi küme sayısını (K) bulmaya çalıştırılıyor.....")
best_k = 3
best_score = -1
scores = []

for k in range(2, 7):
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init='auto')
    labels_temp = kmeans_temp.fit_predict(X_scaled)

    score = silhouette_score(X_scaled, labels_temp, sample_size=1000, random_state=42)
    scores.append((k, score))
    if score > best_score:
        best_score = score
        best_k = k

print(f"En iyi K değeri: {best_k} (Skor: {best_score:.4f})")

start = time.time()
kmeans = KMeans(n_clusters=best_k, random_state=42)
kmeans_labels = kmeans.fit_predict(X_scaled)
results.append(evaluate_clustering(f"K-Means (K={best_k})", kmeans_labels, X_scaled, time.time()-start))

print("MiniBatch K-Means çalıştırılıyor...")
start = time.time()
mb_kmeans = MiniBatchKMeans(n_clusters=4, random_state=42, batch_size=256)
mb_labels = mb_kmeans.fit_predict(X_scaled)
results.append(evaluate_clustering("MiniBatch KMeans", mb_labels, X_scaled, time.time()-start))

print("DBSCAN çalıştırılıyor... (Parametre ayarı kritik)")
start = time.time()

dbscan = DBSCAN(eps=2.0, min_samples=10)
db_labels = dbscan.fit_predict(X_scaled)
res = evaluate_clustering("DBSCAN", db_labels, X_scaled, time.time()-start)
if res: results.append(res)

results_df = pd.DataFrame(results)
print("\nSonuçların karşılaştırılması:")
print(results_df)

try:
    cols = X.columns.tolist()

    x_col = 'Spotify Streams' if 'Spotify Streams' in cols else cols[0]
    y_col = 'TikTok Views' if 'TikTok Views' in cols else (cols[1] if len(cols) > 1 else cols[0])

    plt.figure(figsize=(8, 6))

    sns.scatterplot(x=X[x_col], y=X[y_col], hue=kmeans_labels, palette='viridis', alpha=0.6)
    plt.title(f'{x_col} vs {y_col} (K-Means Kümeleri)')
    plt.xscale('log')
    plt.yscale('log')
    plt.savefig('clustering_scatter_streams.png')
    print(f"Grafik kaydedildi.")
    plt.close()
except Exception as e:
    print(f"Ek grafik çizilirken hata: {e}")

print("\nKümeleme işlemi bitti.")
